# Deploy the App
Use Databricks SDK with notebook authentication.

In [0]:
%pip install PyYAML
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import apps

w = WorkspaceClient()

app_name = "chainlink"

In [0]:
from databricks.sdk.service.apps import App

try:
    existing_app = w.apps.get(name=app_name)
except:
    existing_app = None

if not existing_app:
    print(f'Creating app {app_name}...')
    app = App(name=app_name)
    w.apps.create_and_wait(app=app)
else:
    print(f'App {app_name} already exists.')

In [0]:
import os
from databricks.sdk.service.apps import AppDeployment

app_deployment = AppDeployment(
    source_code_path=os.getcwd() + "/_resources/streamlit"
)

print(f'Deploying app {app_name} from source code...')
deployment = w.apps.deploy(
    app_name=app_name,
    app_deployment=app_deployment
).result()
print('App deployed.')

## Grant App Permission to Serving Endpoint
Need to grant CAN_QUERY permission on endpoint to app service principal

In [0]:
import yaml

def read_config(file_path):
    try:
        with open(file_path, 'r') as file:
            config = yaml.safe_load(file)
            return config
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

config = read_config('./_resources/streamlit/app.yaml')

In [0]:
from databricks.sdk.service.serving import ServingEndpointAccessControlRequest, ServingEndpointPermissionLevel

# Define the serving endpoint ID and permissions
# Note: Using SDK overwrites permissions so you need to make sure the creator still has permission
# (also granting access to person running this notebook)
serving_endpoint_id = w.serving_endpoints.get(name=config.get('env')[0]['value']).id
serving_endpoint_creator = w.serving_endpoints.get(name=config.get('env')[0]['value']).creator
existing_app = w.apps.get(name=app_name)

access_control_list = [
    ServingEndpointAccessControlRequest(
        user_name=serving_endpoint_creator,
        permission_level=ServingEndpointPermissionLevel["CAN_MANAGE"]
    ),
    ServingEndpointAccessControlRequest(
        user_name=w.current_user.me().user_name,
        permission_level=ServingEndpointPermissionLevel["CAN_MANAGE"]
    ),
    ServingEndpointAccessControlRequest(
        user_name=existing_app.service_principal_client_id,
        permission_level=ServingEndpointPermissionLevel["CAN_QUERY"]
    )
]

# Grant permissions to the serving endpoint
w.serving_endpoints.set_permissions(
    serving_endpoint_id=serving_endpoint_id,
    access_control_list=access_control_list
)

print(f"Permissions granted successfully for serving endpoint {serving_endpoint_id}.")